# ０　元ネタ：ここを参考に実装しました
 - Running Flask and FastAPI on Google Colab
  - https://medium.datadriveninvestor.com/flask-on-colab-825d2099d9d8

# １　必要なライブラリのインストール

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

# ２　簡単な例でFastAPIの起動

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

jp_yama = {
    1:'富士山(3,776)',
    2:'北岳(3,193)',
    3:'穂高岳(3,190)',
    4:'槍ヶ岳(3,180)',
    5:'荒川岳(3,141)',
    6:'赤石岳(3,120)',
    7:'御嶽山(3,067)',
    8:'塩見岳(3,047)',
    9:'仙丈ヶ岳(3,033)',
    10:'乗鞍岳(3,026)'    
}

app = FastAPI()

@app.get('/test')
async def get(
        query: int, 
        ):
    yama = jp_yama[query]
    return {"日本の高い山": f"{yama}"}

# トンネリング
ngrok_tunnel = ngrok.connect(8000)
print(f'公開URL: {ngrok_tunnel.public_url}')
print(f'Swagger UI: {ngrok_tunnel.public_url}/docs')
# 非同期通信
nest_asyncio.apply()
# Webサーバ
uvicorn.run(app, port=8000)

# 3　ＡＰＩサービスの実装
## 実装するアプリの概要
- リクエストされたキーワードでＤＢを検索して、レスポンスは検索結果をjson形式で返す。
- 実装する前にテスト
  - ＤＢを検索する
  - 検索結果をjson形式に変換する

## テスト：ＤＢを検索する

### オーソドックスな検索方法

In [ ]:
import sqlite3

PATH = "/content/drive/MyDrive/0_HanPy41/2_APIサービス/report.db"

key_word = "IoT"

conn = sqlite3.connect(PATH)
c = conn.cursor()
SQL = f"SELECT * FROM master where report like '%{key_word}%'"
rows = c.execute(SQL)
data = rows.fetchall()

conn.commit()
conn.close()

data[:2]

### 出力を考慮するとpandas一択

In [ ]:
import pandas as pd
import sqlite3

PATH = "/content/drive/MyDrive/0_HanPy41/2_APIサービス/report.db"
key_word = "IoT"

conn = sqlite3.connect(PATH)
SQL = f"SELECT * FROM master where report like '%{key_word}%'"
df = pd.read_sql(SQL, conn)

df.head(2)

## テスト：検索結果をjson形式に変換する

In [ ]:
json = df.to_json()
json # トラブル発生！　レコード（横）単位で出力したいのに、項目（縦）単位で出力される。

### 困ったらググる
- 例：pandas dataframe to_json レコード単位
 - https://www.google.com/search?q=pandas+dataframe+to_json+%E3%83%AC%E3%82%B3%E3%83%BC%E3%83%89%E5%8D%98%E4%BD%8D&oq=pandas+dataframe+to_json+%E3%83%AC%E3%82%B3%E3%83%BC%E3%83%89%E5%8D%98%E4%BD%8D&aqs=chrome..69i57.911j0j15&sourceid=chrome&ie=UTF-8

### 解決策の確認

In [ ]:
json = df.to_json(orient='records')
json

## いざ実装

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import pandas as pd
import sqlite3

PATH = "/content/drive/MyDrive/0_HanPy41/2_APIサービス/report.db"

app = FastAPI()

@app.get('/api')
async def get(
        key_word: str, 
        ):
    conn = sqlite3.connect(PATH)
    SQL = f"SELECT * FROM master where report like '%{key_word}%'"
    df = pd.read_sql(SQL, conn)
    return df.to_json(orient='records')

# トンネリング
ngrok_tunnel = ngrok.connect(8000)
print(f'公開URL: {ngrok_tunnel.public_url}')
print(f'Swagger UI: {ngrok_tunnel.public_url}/docs')
# 非同期通信
nest_asyncio.apply()
# Webサーバ
uvicorn.run(app, port=8000)